# Set Cover Problem

The set cover problem is a combinatorial optimization problem. The problem is to find the smallest set of sets that covers all elements of a given universe. For example, suppose we have a universe of 10 elements and 5 sets, each of which contains a subset of the universe. The goal is to find the smallest set of sets that covers all elements of the universe. 

Since the problem is NP-hard, we have to use a heuristic algorithm to solve and there exists no theoretically polynomial time algorithm.

In this notebook, I will attempt to solve the set cover problem using the following algorithms:
1. Naive Greedy
2. Greedy with a better cost function
3. Building a Fully Connected Graph and over-complicating things
2. Breadth First Traversal
3. A* Traversal

> Sidharrth Nagappan, 2022

### Necessary Imports

In [1]:
import random
from collections import deque
import itertools
import time

### Building the problem set

In [2]:
# setting a constant seed for reproducibility
SEED = 42

def problem(N, seed=SEED):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

for n in [5]:
    print(f"N = {problem(n)}")

N = [[0], [1], [0], [4], [0], [1], [4], [4], [4], [1, 3], [0, 1], [2], [1], [0], [0, 2], [2, 4], [3], [3], [4], [2, 4], [0], [1], [0, 1], [3], [2, 3]]


### Naive Greedy Algorithm

The greedy algorithm essentially traverses through a sorted list of subsets and keeps adding the subset to the solution set if it covers any new elements. The algorithm is very naive as it does not take into account the number of new elements.

In [3]:
def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    print(
        f"Naive greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

### Smarter Greedy Algorithm (Sorting at each step)

In real-life scenarios, the cost depends on the relative price of visiting a node/choosing an option. Since we consider all options to be arbitrarily priced, we use a constant cost of 1. This version of the greedy algorithm takes the subset with the lowest $f$ where:

- $S_e$ is the expected solution (containing all the unique elements)
- $n_i$ is the current subset
- The cost is set to 1 here, since there is no "business" cost associated with choosing a subset

$$f_i = 1 / |n_i - S_e|$$

In [4]:
def set_covering_problem_greedy(expected_solution, subsets, costs):
    cost = 0
    visited_nodes = 0
    already_discovered = set()
    final_solution = []
    while covered != expected_solution:
        subset = min(subsets, key=lambda s: costs[subsets.index(s)] / (len(s-covered) + 1))
        final_solution.append(subset)
        cost += costs[subsets.index(subset)]
        visited = visited+1
        covered |= subset
    print("NUMBER OF VISITED NODES: ", visited)
    print("w: ", sum(len(_) for _ in final_solution))
    return final_solution, cost

### A* Traversal

The A* algorithm requires a monotonic heuristic function that symbolises the remaining distance between the current state and the goal state. In the case of the set cover problem, the heuristic function is the number of elements that are not covered by the current solution set. The algorithm is implemented using a priority queue. There are two ways of implementing A*, we can either:

1. build a fully connected graph and use an open and closed list to traverse
2. use a priority queue

For learning purposes, I will implement both, even though the second method is more efficient.

#### A* Traversal Using Priority Queue

In [16]:
from typing import Callable
from helpers import State, PriorityQueue
import numpy as np

class AStarSearch:
    def __init__(self, N, seed=42):
        # N is the number of elements to expect
        self.N = N
        self.seed = seed
    
    def add_to_state(self, st, subset):
        '''
        Unnecessary function to add a subset to a state because we are using the State class instead of a normal np.array
        '''
        state_list = st.copy_data().tolist()
        state_list.append(subset)
        return State(np.asarray(state_list, dtype=object))

    def are_we_done(self, state):
        '''
        Check if we have reached the goal state (such that all elements are covered in range(N))
        '''
        flattened_list = self.flatten_list(state.copy_data().tolist())
        for i in range(self.N):
            if i not in flattened_list:
                return False
        # print("We are done")
        return True
    
    def flatten_list(self, l):
        '''
        Utility function to flatten a list of lists using itertools
        '''
        return list(itertools.chain.from_iterable(l))
    
    def h(self, state):
        '''
        Heuristic Function h(n) = number of undiscovered elements
        '''
        num_undiscovered_elements = len(set(range(self.N)) - set(self.flatten_list(state.copy_data().tolist())))
        return num_undiscovered_elements

    def astar_search(
        self,
        initial_state: State,
        subsets: list,
        parents: dict,
        cost_of_each_state: dict,
        priority_function: Callable,
        unit_cost: Callable,
    ):
        frontier = PriorityQueue()
        parents.clear()
        cost_of_each_state.clear()
        
        state = initial_state
        parents[state] = None
        cost_of_each_state[state] = 0
        # to find length at the end without needed to flatten the state
        discovered_elements = []
        
        while state is not None and not self.are_we_done(state):
            for subset in subsets:
                # if this list has already been collected, skip
                if subset in state.copy_data():
                    # print("Already in")
                    continue
                new_state = self.add_to_state(state, subset)
                state_cost = unit_cost(subset)
                # if new_state not in cost_of_each_state or cost_of_each_state[new_state] > cost_of_each_state[state] + state_cost:
                if new_state not in cost_of_each_state and new_state not in frontier:
                    parents[new_state] = state
                    cost_of_each_state[new_state] = cost_of_each_state[state] + state_cost
                    frontier.push(new_state, p=priority_function(new_state))
                elif new_state in frontier and cost_of_each_state[new_state] > cost_of_each_state[state] + state_cost:
                    parents[new_state] = state
                    cost_of_each_state[new_state] = cost_of_each_state[state] + state_cost
            if frontier:
                state = frontier.pop()
            else:
                state = None

        path = list()
        s = state

        while s:
            path.append(s.copy_data())
            s = parents[s]

        print(f"Length of final list: {len(self.flatten_list(path[0]))}")
        print(f"Found a solution in {len(path):,} steps; visited {len(cost_of_each_state):,} states")
        print(
            f"My solution for N={self.N}: w={sum(len(_) for _ in path[0])} (bloat={(sum(len(_) for _ in path[0])-self.N)/self.N*100:.0f}%)"
        )
        return list(reversed(path))
    
    def search(self):
        GOAL = State(np.array(range(self.N)))
        subsets = problem(self.N, seed=self.seed)
        initial_state = State(np.array([subsets[0]]))

        parents = dict()
        cost_of_each_state = dict()

        self.astar_search(
            initial_state = initial_state,
            subsets = subsets,
            parents = parents,
            cost_of_each_state = cost_of_each_state,
            priority_function = lambda state: cost_of_each_state[state] + self.h(state),
            unit_cost = lambda subset: len(subset),
        )

In [18]:
for n in [5, 10, 20]:
    print(f"N = {n}")
    engine = AStarSearch(N=n, seed=42)
    engine.search()

N = 5
Length of final list: 5
Found a solution in 4 steps; visited 59 states
My solution for N=5: w=5 (bloat=0%)
N = 10
Length of final list: 10
Found a solution in 4 steps; visited 141 states
My solution for N=10: w=10 (bloat=0%)
N = 20


<ipython-input-16-63b75c29188a>:65: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if subset in state.copy_data():


Length of final list: 23
Found a solution in 5 steps; visited 52,286 states
My solution for N=20: w=23 (bloat=15%)


#### A-Star Traversal Using a Fully Connected Graph

Creating a fully-connected graph and traversing through it. In this case, we also use a more advanced heuristic function that divides the length of the subset by the number of discovered elements (a.k.a the value) from that subset. This is slightly faster than the solution above, but does not always find an optimal or better solution than the ones above. Effectively, this is a work in progress.

Part of the main a-star traversal code was adapted from stackabuse.com.

In [13]:
class AStarSearchFullyConnectedGraph:
    def __init__(self, adjacency_list, list_values, N):
        self.adjacency_list = adjacency_list
        self.list_values = list_values
        H = {}
        for key in list_values:
            # heuristic value is length of list
            H[key] = len(list_values[key])
        self.H = H
        # holds the lists of each visited node
        self.final_list = []
        # N is the count of elements that should be in the final list
        self.N = N
        self.discovered_elements = set()

    def flatten_list(self, _list):
        return list(itertools.chain.from_iterable(_list))

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    def get_number_of_elements_not_in_second_list(self, list1, list2):
        count = 0
        # flattened_list = self.flatten_list(list2)
        for i in set(list1):
            # print("i: ", i)
            if i not in list2:
                count += 1
        # if count > 1:
        #     print("count: ", count)
        return len(set(list1) - set(list2))

    # f(n) = h(n) + g(n)

    def h(self, n):
        num_new_elements = self.get_number_of_elements_not_in_second_list(self.list_values[n], self.discovered_elements)
        # if self.list_values[n] in self.final_list:
        #     return 1000
        return num_new_elements
        # return self.H[n] / (num_new_elements + 1)

    def get_node_with_least_h(self):
        min_h = float("inf")
        min_node = None
        for node in self.adjacency_list:
            if self.h(node) < min_h:
                min_h = self.h(node)
                min_node = node
        return min_node

    def get_node_with_least_h_and_not_in_final_list(self):
        min_h = float("inf")
        min_node = None
        for node in self.adjacency_list:
            if self.h(node) < min_h and node not in self.final_list:
                min_h = self.h(node)
                min_node = node
        return min_node

    # visited_node = [1, 2, 3]
    # final_list = [[4, 5], [1]]
    def are_we_done(self):
        # flattened_list = list(itertools.chain.from_iterable(self.final_list))
        for i in range(self.N):
            if i not in self.discovered_elements:
                return False
        print("We are done")
        return True

    def insert_unique_element_into_list(self, _list, element):
        if element not in _list:
            _list.append(element)
        return _list

    def a_star_algorithm(self):
        # start_node is node with lowest cost
        start_node = self.get_node_with_least_h()

        open_list = [start_node]
        closed_list = []

        g = {}

        g[start_node] = 0

        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None

            # find a node with the highest value of f() - evaluation function
            for v in open_list:
                if n == None or g[v] + self.h(v) > g[n] + self.h(n):
                    n = v;

            if n == None:
                print('Path does not exist!')
                return None

            print(f"Visiting node: {n}")
            self.final_list.append(self.list_values[n])
            # self.discovered_elements.union(self.list_values[n])
            # add list_values[n] to discovered_elements
            for i in self.list_values[n]:
                self.discovered_elements.add(i)
            print(len(self.discovered_elements))

            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if self.are_we_done():
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print(f"Number of elements in final list: {len(self.flatten_list(self.final_list))}")
                print('Path found: {}'.format(reconst_path))
                print(
                    f"My solution for N={N}: w={sum(len(_) for _ in self.final_list)} (bloat={(sum(len(_) for _ in self.final_list)-N)/N*100:.0f}%)"
                )
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                values = self.list_values[m]
                if m not in open_list and m not in closed_list:
                    # open_list.add(m)
                    open_list = self.insert_unique_element_into_list(open_list, m)
                    # sort open_list by self.h
                    open_list = sorted(open_list, key=self.h)
                    parents[m] = n
                    g[m] = g[n] + weight

                else:
                    if g[m] + self.h(m) > g[n] + self.h(n) + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        # if m in closed_list:
                        #     closed_list.remove(m)
                        #     # open_list.add(m)
                        #     open_list = self.insert_unique_element_into_list(open_list, m)
                        #     open_list = sorted(open_list, key=self.h)


            open_list.remove(n)
            open_list = sorted(open_list, key=self.h)
            closed_list = self.insert_unique_element_into_list(closed_list, n)

        print('Path does not exist!')
        return None
